# 10 categorías que tienen la menor cantidad de contenido anónimo publicado. (⭐)

In [17]:
import pyspark as sp

In [18]:
spark = sp.sql.SparkSession.builder.getOrCreate()
category = spark.read.csv('../data/categorylinks.csv', header=True, inferSchema=True).rdd
contents = spark.read.csv('../data/contents.csv', header=True, inferSchema=True).rdd

In [19]:
# Contents
#  (1, 'id'),
#  (2, 'title'),
#  (3, 'namespace'),
#  (4, 'revision_id'),
#  (5, 'revision_timestamp'),
#  (6, 'revision_username'),
#  (7, 'revisor_id'),
#  (8, 'revisor_ip'),
#  (9, 'revision_comment')]

In [20]:
# Category
# [(0, 'cl_from'),
#  (1, 'cl_to'),
#  (2, 'cl_sortkey'),
#  (3, 'cl_timestamp'),
#  (4, 'cl_sortkey_prefix'),
#  (5, 'cl_collation'),
#  (6, 'cl_type')]

In [21]:
# Filtro contents y transformo sus registros a tuplas (content_id, None).
contents = contents.filter(lambda x: not x[6] and x[8]).map(lambda x: (x[1], None))
# Me hubiera gustado no guardarme un campo None pero no
# supe como hacer un join sin tener una tupla key value.

In [22]:
# i)   Transformo los registros de category a tuplas (content_id, nombre_de_su_categoria).
# ii)  Junto los dos RDDs por su content_id consiguiendo los registros (content_id, (nombre_de_su_categoria, None)).
# iii) Transformo los registros a tuplas (nombre_de_categoria, 1).
# iv)  Sumo sus valores por clave.
# v)   Me quedo con las 10 categorias con menos articulos.
category.map(lambda x: (x[0], x[1]))\
    .join(contents)\
    .map(lambda x: (x[1][0], 1))\
    .reduceByKey(lambda x,y: x+y)\
    .takeOrdered(10, key=lambda x: x[1])


[('1893', 1),
 ('Plantas_descritas_en_1994', 1),
 ('Trabajo_en_JapÃ³n', 1),
 ('Chile_en_1862', 1),
 ('Batallas_en_la_provincia_de_Ciudad_Real', 1),
 ('IntÃ©rpretes_de_mÃºsica_clÃ¡sica_de_Venezuela', 1),
 ('Fascismo_en_EspaÃ±a', 1),
 ('Localidades_establecidas_en_1580', 1),
 ('AerolÃ\xadneas_desaparecidas_en_2010', 1),
 ('Constitucionalistas_de_PanamÃ¡', 1)]